# Sisal Selenium Scraper Test

This notebook tests the Selenium-based scraper with a specific live match: **Francia U21 vs Georgia U21**.

The scraper will connect to the Sisal page, extract betting odds, and create a BettingOdds instance.

In [1]:
import sys
import os

# Add the project root to Python path
# Get the current notebook directory and go up one level to reach project root
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else 'scraper_test.ipynb'))
project_root = os.path.dirname(current_dir)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import the Selenium scraper class
from src.scraper import SisalSeleniumScraper

In [ ]:
https://www.sisal.it/scommesse-live/evento/calcio/cecafa-cup-f/burundi-kenya

In [ ]:
# Test scraper with the specified match
# Target match URL
url = "https://www.sisal.it/scommesse-live/evento/calcio/cecafa-cup-f/burundi-kenya"
print("Testing Sisal Selenium Scraper")
print(f"URL: {url}")
print("=" * 60)

# Create scraper instance and scrape the odds
scraper = SisalSeleniumScraper(headless=False)
odds = scraper.scrape_betting_odds(url)

if odds:
    print("✅ SUCCESS! Betting odds extracted")
    print(f"Match: {odds.home_team} vs {odds.away_team}")
    print(f"Timestamp: {odds.timestamp}")
    print(f"Source: {odds.source}")
    print()
    
    # Display odds
    print("Extracted odds:")
    print(f"  1X2: {odds.home_win} / {odds.draw} / {odds.away_win}")
    print(f"  O/U 2.5: {odds.over_2_5} / {odds.under_2_5}")
    print(f"  BTTS: {odds.both_teams_score_yes} / {odds.both_teams_score_no}")
    print(f"  Double Chance: {odds.home_or_draw} / {odds.away_or_draw} / {odds.home_or_away}")
    print()
    
    # Show complete BettingOdds object
    print("Complete BettingOdds object:")
    print(repr(odds))
    
else:
    print("❌ Failed to extract odds")
    print("Possible reasons:")
    print("- Match not live")
    print("- Geographic restrictions") 
    print("- Page structure changed")
    print("- Betting markets not available")

Testing Sisal Selenium Scraper
URL: https://www.sisal.it/scommesse-live/evento/calcio/torneo-di-tolone/denmark-youth-messico
Starting scraping for: https://www.sisal.it/scommesse-live/evento/calcio/torneo-di-tolone/denmark-youth-messico
✓ CSV storage initialized: data\sisal_odds_20250615_151040.csv
✓ Chrome WebDriver setup successful
✓ Cookie banner accepted
✓ Page content loaded
✓ Teams: Denmark Youth vs Messico
✓ Expanded betting section
✓ Expanded betting section

=== EXTRACTED BETTING ODDS ===
Match: Denmark Youth vs Messico
Match ID: denmark-youth-messico
Source: Sisal
Timestamp: 2025-06-15 15:10:49.058106

✓ Stored odds for Denmark Youth vs Messico
✓ Browser closed
✅ SUCCESS! Betting odds extracted
Match: Denmark Youth vs Messico
Timestamp: 2025-06-15 15:10:49.058106
Source: Sisal

Extracted odds:
  1X2: None / None / None
  O/U 2.5: None / None
  BTTS: None / None
  Double Chance: None / None / None

Complete BettingOdds object:
BettingOdds(timestamp=datetime.datetime(2025, 6, 1